# Задание 1

<br>
Есть таблица с действиями юзеров A/B-теста (<i>task_1_events.csv</i>):

- $events$ - установки приложения
    - $user\_id$ - id юзера,
    - $ab\_group$ - группа A/B-теста,
    - $ts$ - время совершения действия,
    - $pdate$ - дата совершения действия.

Пользовательская сессия определяется по следующим правилам:
<br>
1. Новая сессия начинается после 30 минут бездействия.
<br>
2. Сессия прерывается при переходе между двумя датами.

<b>Постройте таблицу с сессиями юзеров в формате<b>:
<br>
- $user\_id$ - id юзера
- $ab\_group$ - группа A/B-теста,
- $start\_ts$ - время старта сессии,
- $end\_ts$ - время окончания сессии,
- $pdate$ - дата сессии.

# Задание 2

Был проведен A/B-тест.
<br>
В качестве данных используйте таблицу, построенную в предыдущем задании. Первая сессия юзера считается моментом попадания в A/B-тест.
<br><br>
Ключевая метрика эксперимента - конверсия во вторую сессию.
<br>
Сделайте вывод о том, какая группа выиграла в A/B-тесте. Ответ обоснуйте.
<br>

# Этап 1. Получение данных

In [48]:
# Импортируем необходимые библиотеки 
import pandas as pd
import numpy as np

In [49]:
# Читаем файл и сохраняем его в df
df = pd.read_csv('./task_1_events.csv')

In [50]:
# Смотрим первые 10 записей
df.head(10)

,user_id,ab_group,ts,pdate
0,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:24:33.553327,2023-05-02
1,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:32:23.215284,2023-05-02
2,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:33:01.050395,2023-05-02
3,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:36:18.233874,2023-05-02
4,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:36:45.210274,2023-05-02
5,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:36:54.251438,2023-05-02
6,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:37:39.181624,2023-05-02
7,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:37:57.761142,2023-05-02
8,3E69ZO0RJrEWACi28XaTPg==,A,2023-05-01 18:58:45.338597,2023-05-01
9,3E69ZO0RJrEWACi28XaTPg==,A,2023-05-01 19:01:05.548678,2023-05-01


### Общая информация о данных

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44568 entries, 0 to 44567
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   44568 non-null  object
 1   ab_group  44568 non-null  object
 2   ts        44568 non-null  object
 3   pdate     44568 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


#### Всего 4 колонки. Тип данных у всех object

# Этап 2. Предобработка данных

#### Проверим, есть ли пропуски в данных

In [52]:
df.isnull().sum() ### Пропусков нет

user_id     0
ab_group    0
ts          0
pdate       0
dtype: int64

#### Поменяем типы данных pdate и ts с object на date

In [53]:
df['pdate'] = pd.to_datetime(df['pdate'])

In [54]:
df['ts'] = pd.to_datetime(df['ts'], format='%Y-%m-%d %H:%M:%S.%f')

In [55]:
df.dtypes

user_id             object
ab_group            object
ts          datetime64[ns]
pdate       datetime64[ns]
dtype: object

# Этап 3. Создаем таблицу с сессиями юзеров

In [56]:
df = df.sort_values(['user_id', 'ts']) 
df['time_diff'] = df.groupby('user_id')['ts'].diff().fillna(pd.Timedelta(seconds=0))
# Считаем разницу во времени для каждого пользователя, пропуски заполняем 0

In [57]:
df.head(5)

,user_id,ab_group,ts,pdate,time_diff
22360,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:45.766570,2023-05-01,0 days 00:00:00
22361,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:23.403884,2023-05-01,0 days 00:00:37.637314
22362,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:24.197308,2023-05-01,0 days 00:00:00.793424
22363,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:26.551874,2023-05-01,0 days 00:00:02.354566
22364,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:28.284787,2023-05-01,0 days 00:00:01.732913


In [58]:
session_threshold = pd.Timedelta(minutes=30) # Новая сессия начинается через 30 минут
session_mask = ((df['time_diff'] > session_threshold) | (df['pdate'].diff().dt.days > 0)) # Проверяем, прошло бодьше 30 мин или 
# нет и произошло ли изменение даты
df['session_id'] = session_mask.groupby(df['user_id']).cumsum() + 1  # Создаем колонку для нумерации сессий, для каждого
# пользователя отсчет сессий начинается с 1

In [59]:
df.head(10)

,user_id,ab_group,ts,pdate,time_diff,session_id
22360,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:45.766570,2023-05-01,0 days 00:00:00,1
22361,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:23.403884,2023-05-01,0 days 00:00:37.637314,1
22362,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:24.197308,2023-05-01,0 days 00:00:00.793424,1
22363,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:26.551874,2023-05-01,0 days 00:00:02.354566,1
22364,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:28.284787,2023-05-01,0 days 00:00:01.732913,1
22365,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:54.868559,2023-05-01,0 days 00:00:26.583772,1
22366,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:20.092009,2023-05-01,0 days 00:04:25.223450,1
22367,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:20.688954,2023-05-01,0 days 00:00:00.596945,1
22368,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:21.359983,2023-05-01,0 days 00:00:00.671029,1
22369,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:23:23.231156,2023-05-01,0 days 00:00:01.871173,1


In [60]:
# Создаем новую таблицу 
sessions_df = df.groupby(['user_id', 'session_id']).agg(
    ab_group=('ab_group', 'first'), # Группа теста А/В
    start_ts=('ts', 'min'), # Начало сессии
    end_ts=('ts', 'max'), # Конец сессии
    date=('pdate', 'first') # Дата
).reset_index()

In [61]:
sessions_df.sort_values('user_id')

,user_id,session_id,ab_group,start_ts,end_ts,date
0,2JM4L+sRXYIWAFSC8XaTPg==,1,B,2023-05-01 08:17:45.766570,2023-05-01 08:57:37.363424,2023-05-01
1,2OgK/ukRgvkWAOaa8XaTPg==,2,A,2023-05-02 12:24:33.553327,2023-05-02 12:37:57.761142,2023-05-02
2,3E69ZO0RJrEWACi28XaTPg==,1,A,2023-05-01 18:58:45.338597,2023-05-01 19:04:19.203838,2023-05-01
3,3E69ZO0RJrEWACi28XaTPg==,2,A,2023-05-04 10:01:39.700496,2023-05-04 10:07:39.858666,2023-05-04
4,3GsYmewRRswWABqG8XaTPg==,1,B,2023-05-03 05:00:41.797222,2023-05-03 05:18:12.137282,2023-05-03
...,...,...,...,...,...,...
5919,xppk/u0RbGIWAKC+6VfbPg==,3,A,2023-05-03 05:47:34.156020,2023-05-03 05:54:55.011521,2023-05-03
5920,xppk/u0RbGIWAKC+6VfbPg==,4,A,2023-05-05 07:25:25.292417,2023-05-05 07:54:33.997134,2023-05-05
5921,xppk/u0RbGIWAKC+6VfbPg==,5,A,2023-05-06 10:45:48.295508,2023-05-06 10:53:47.921312,2023-05-06
5922,xppk/u0RbGIWAKC+6VfbPg==,6,A,2023-05-07 15:53:11.323273,2023-05-07 16:01:30.187277,2023-05-07


### Участники теста

In [62]:
sessions_df.groupby('ab_group')['session_id'].count()

ab_group
A    2947
B    2977
Name: session_id, dtype: int64

### Пользователи, которые перешли во вторую сессию

In [63]:
sessions_df.groupby('ab_group')['session_id'].apply(lambda x: (x > 1).sum())

ab_group
A    925
B    954
Name: session_id, dtype: int64

### Конверсия для группы A = 925/2947 = 31,4%
### Конверсия для группы В = 954/2977 = 32%

### Пользователи, у которых была только одна сессия

In [64]:
sessions_df.groupby('ab_group')['session_id'].apply(lambda x: (x <= 1).sum())

ab_group
A    2022
B    2023
Name: session_id, dtype: int64

# Вывод

### Группа B имеет чуть более высокую конверсию во вторую сессию по сравнению с группой A. Но для более надежных выводов об успешности A/B-теста  важно учитывать статистическую значимость результатов и проводить статистический анализ, чтобы удостовериться, что различия между группами статистически значимы.